<a href="https://colab.research.google.com/github/Saloni0512/PyTorch-practice-/blob/main/PyTorch_model_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
import matplotlib.pyplot as plt

# PyTorch custom datasets





In [ ]:
# Setting up device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

## 1. Get data
here the dataset is a subset of the Food101 dataset.

It contains only 10% of the images (~75 training and ~25 testing)

In [ ]:
import requests
import zipfile
from pathlib import Path

# Setup path to a data folder
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

# If the image folder doesnt exist, download and prepare it
if image_path.is_dir():
  print(f"{image_path} directory already exists")
else:
  print(f"{image_path} directory doesnt exist... creating one")
  image_path.mkdir(parents=True, exist_ok=True)

# Download the data
with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
  request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
  f.write(request.content)

# Unzip the data
with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
  zip_ref.extractall(image_path)


## 2. Data prep and exploration

In [ ]:
import os
def walk_through_dir(dir_path):
  """Displays the brief overview of a given directory."""
  for dirpath, dirnames, filenames in os.walk(dir_path):
    print(f"there are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'")

walk_through_dir(image_path)

In [ ]:
# Setting up training and testing paths
train_dir = image_path / "train"
test_dir = image_path / "test"

train_dir, test_dir

### 2.1 Visualising the images

In [ ]:
import random
from PIL import Image

# Set the seed
random.seed(42)

image_path_list = list(image_path.glob("*/*/*.jpg"))

random_image_path = random.choice(image_path_list)

# image class is the name of the directory in which image is stored
image_class = random_image_path.parent.stem

# open image
img = Image.open(random_image_path)

# print metadata
print(f"Random image path: {random_image_path}")
print(f"image class: {image_class}")
print(f"image height: {img.height}")
print(f"image width: {img.width}")
img

In [ ]:
import numpy as np

img_as_array = np.asarray(img)

plt.figure(figsize=(10,7))
plt.imshow(img_as_array)
plt.title(f"Image class: {image_class} | Image shape: {img_as_array.shape} -> [height,width,color_channels]")
plt.axis(False);

In [ ]:
print(img_as_array)

## 3. Transforming data into tensors

In [ ]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [ ]:
transform_data = transforms.Compose([
    transforms.Resize(size=(64,64)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor() # turns the image into torch.Tensor
])

In [ ]:
transform_data(img)

In [ ]:
transform_data(img).shape

In [ ]:
# Let's visualise the transformed images
def plot_transformed_image(image_paths: list, transform, n=3, seed=None):
  if seed:
    random.seed(seed)
  random_image_paths = random.sample(image_paths, k=n)
  for image_path in random_image_paths:
    with Image.open(image_path) as f:
      fig, ax = plt.subplots(nrows=1, ncols=2)
      ax[0].imshow(f)
      ax[0].set_title(f"Original\nSize: {f.size}")
      ax[0].axis(False)

      transformed_image = transform(f).permute(1,2,0) # matplotlib expects the HWC format, so we use permute here
      ax[1].imshow(transformed_image)
      ax[1].set_title(f"Transformed\nSize: {transformed_image.shape}")
      ax[1].axis("off")

      fig.suptitle(f"Class: {image_path.parent.stem}", fontsize=14)

plot_transformed_image(image_path_list,
                       transform=transform_data,
                       n=3,
                       seed=None)

## 4. Option1: Loading image data using `ImageFolder`
ImageFolder is a prebuilt datasets function

In [ ]:
# Use ImageFolder to create dataset(s)
train_data = datasets.ImageFolder(root=train_dir,
                                   transform=transform_data,
                                   target_transform=None)

test_data = datasets.ImageFolder(root=test_dir,
                                 transform=transform_data,
                                 target_transform=None)

train_data, test_data

In [ ]:
class_names = train_data.classes
class_names

In [ ]:
class_dict = train_data.class_to_idx
class_dict

In [ ]:
train_data.samples[0]

In [ ]:
# Index on the train data to get a single image and its label
img, label = train_data[0][0] , train_data[0][1]
print(f"Image tensor: \n{img}")
print(f"Image shape: {img.shape}")
print(f"Image label: {label}")
print(f"Image class: {class_names[label]}")

### 4.1 Turn loaded image data into `DataLoader`
A dataloader helps us to turn Dataset into iterables

In [ ]:
# Lets turn train data and test data into DataLoader
BATCH_SIZE = 1 # hyperparam
train_dataloader = DataLoader(dataset=train_data,
                              batch_size=BATCH_SIZE,
                              num_workers=1, # no of CPU cores used to load the data , use os.cpu_count() as value if more cpus needed
                              shuffle=True)

test_dataloader = DataLoader(dataset=test_data,
                             batch_size=BATCH_SIZE,
                             num_workers=1,
                             shuffle=False)

train_dataloader, test_dataloader

In [ ]:
len(train_dataloader), len(test_dataloader)

In [ ]:
img, label = next(iter(train_dataloader))
img.shape, label.shape

## 5. Option 2: Loading image data using a Custom `Dataset`
* load images from a file
* get class names from the dataset
* get classes as dictionary from the dataset
>Note: We can create a `Dataset` almost out of anything, but it also means writing more code that could have errors or lead to performance issues



In [ ]:
from torch.utils.data import Dataset
from typing import Tuple, Dict, List

### 5.1 Get class names from the directory

In [ ]:
# create a target directory
target_dir = train_dir

# Get class names from the target directory
class_names_found = sorted([entry.name for entry in list(os.scandir(target_dir))])
class_names_found

In [ ]:
list(os.scandir(target_dir))

In [ ]:
# A helper function that we will use in our custom dataset
def find_classes(directory: str) -> Tuple[List[str], Dict[str, int]]:
  """Find class folder names in the target directory"""
  classes = sorted(entry.name for entry in os.scandir(directory) if entry.is_dir())

  classes_to_idx = {class_name: idx for idx, class_name in enumerate(classes)}

  return classes, classes_to_idx

In [ ]:
find_classes(target_dir)

### 5.2 Create a Custom `Dataset` to replicate `ImageFolder`

In [ ]:
# Write a custom dataset class
import pathlib
class ImageFolderCustom(Dataset):
  # Initialise the custom dataset
  def __init__(self,
               targ_dir: str,
               transform=None):
    self.paths = list(pathlib.Path(targ_dir).glob("*/*.jpg"))
    self.transform = transform
    self.classes, self.class_to_idx = find_classes(targ_dir)

  def load_image(self, index: int) -> Image.Image:
    """Opens an image via path and returns it."""
    image_path = self.paths[index]
    return Image.open(image_path)

  def __len__(self) -> int:
    """Returns the total number of samples."""
    return len(self.paths)

  def __getitem__(self, index: int) -> Tuple[torch.Tensor, int]:
    """Returns one sample of data, data and label (X, y)."""
    img = self.load_image(index)
    class_name = self.paths[index].parent.name
    class_to_idx = self.class_to_idx[class_name]

    if self.transform:
      return self.transform(img), class_to_idx # transformed img and label
    else:
      return img, class_to_idx # untransformed img and label


In [ ]:
# Create a transform
train_transforms = transforms.Compose([
    transforms.Resize(size=(64,64)),
    transforms.RandomHorizontalFlip(p=0.5), # data augmentation technique used to increase diversity of training data
    transforms.ToTensor()
])

test_transforms = transforms.Compose([
    transforms.Resize(size=(64,64)),
    transforms.ToTensor()
])

In [ ]:
# Test the ImageFolderCustom
train_data_custom = ImageFolderCustom(targ_dir=train_dir,
                                      transform=train_transforms)

test_data_custom = ImageFolderCustom(targ_dir=test_dir,
                                    transform=test_transforms)
train_data_custom, test_data_custom

In [ ]:
len(train_data_custom), len(test_data_custom)

In [ ]:
train_data_custom.classes , test_data_custom.classes

In [ ]:
train_data_custom.class_to_idx

In [ ]:
train_data_custom.load_image(1), test_data_custom.load_image(1)

In [ ]:
train_data_custom.transform

### 5.3 Create a function to display random images


In [ ]:
def display_random_images(dataset: torch.utils.data.Dataset,
                          classes: List[str]= None,
                          n : int = 10, # no of images to display
                          display_shape: bool = True,
                          seed: int = None):
  if n > 10:
    n= 10
    display_shape = False
    print(f"For display purposes, n shouldn't be larger than 10, setting n = 10 and display_shape = False")

  if seed:
    random.seed(seed)

  random_sample_idxs = random.sample(range(len(train_data_custom)), k=n)
  plt.figure(figsize=(16,8))

  for i, targ_sample in enumerate(random_sample_idxs):
    targ_img, targ_label = dataset[targ_sample][0], dataset[targ_sample][1]
    targ_img_adjust = targ_img.permute(1,2,0) # matplotlib works with HWC order
    # Plot adjusted samples
    plt.subplot(1,n,i+1)
    plt.imshow(targ_img_adjust)
    if classes:
      title = f"class: {classes[targ_label]}"
      if display_shape:
        title = title + f"\nshape: {targ_img_adjust.shape}"
    plt.title(title)
    plt.axis("off")


In [ ]:
# Show random images from ImageFolderCustom dataset
display_random_images(train_data_custom,
                      class_names,
                      n=15,
                      seed=None)

### 5.4 Turn image data from custom dataset into dataloader

In [ ]:
BATCH_SIZE = 1 # hyperparam
train_custom_dataloader = DataLoader(dataset=train_data_custom,
                              batch_size=BATCH_SIZE,
                              num_workers=1, # no of CPU cores used to load the data , use os.cpu_count() as value if more cpus needed
                              shuffle=True)

test_custom_dataloader = DataLoader(dataset=test_data_custom,
                             batch_size=BATCH_SIZE,
                             num_workers=1,
                             shuffle=False)

train_custom_dataloader, test_custom_dataloader

In [ ]:
img, label = next(iter(train_custom_dataloader))
img.shape, label.shape

## 6. Experimenting with data augmentation (other forms of transforms)
Data augmentation is the process of artifically adding diversity to training data. This helps us to view images from different angles and perspectives.

In [ ]:
# 1. TrivialAugment

train_transforms = transforms.Compose([
    transforms.Resize(size=(128,128)),
    transforms.TrivialAugmentWide(num_magnitude_bins=31),
    transforms.ToTensor()
])

test_transforms = transforms.Compose([
    transforms.Resize(size=(128,128)),
    transforms.ToTensor()
])

In [ ]:
plot_transformed_image(image_path_list,
                       transform=train_transforms,
                       n=3,
                       seed=None)

## 7. Model 0: TinyVGG without data augmentation

### 7.1 Load data and create transforms

In [ ]:
# Create transform
simple_transform = transforms.Compose([
    transforms.Resize(size=(64,64)),
    transforms.ToTensor()
])

In [ ]:
# Load and transform data
train_data_simple = datasets.ImageFolder(root=train_dir,
                                         transform=simple_transform,
                                         target_transform=None)

test_data_simple = datasets.ImageFolder(root=train_dir,
                                         transform=simple_transform,
                                         target_transform=None)

# Turn data into dataloader
BATCH_SIZE = 32
NUM_WORKERS = os.cpu_count()

torch.manual_seed(42)
train_dataloader_simple = DataLoader(dataset=train_data_simple,
                                     batch_size=BATCH_SIZE,
                                     num_workers=NUM_WORKERS,
                                     shuffle=True)

test_dataloader_simple = DataLoader(dataset=test_data_simple,
                                     batch_size=BATCH_SIZE,
                                     num_workers=NUM_WORKERS,
                                     shuffle=False)

train_dataloader_simple, test_dataloader_simple

In [ ]:
len(train_dataloader_simple), len(test_dataloader_simple)

In [ ]:
img, label = next(iter(train_dataloader_simple))
img.shape, label.shape

### 7.2 Create TinyVGG model class

In [ ]:
class TinyVGG(nn.Module):
  def __init__(self,
               input_shape: int,
               hidden_units: int,
               output_shape: int) -> None:
    super().__init__()
    self.conv_block_1 = nn.Sequential(
        nn.Conv2d(in_channels=input_shape,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,
                     stride=2)
    )
    self.conv_block_2 = nn.Sequential(
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,
                     stride=2)
    )
    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=hidden_units*13*13,
                  out_features=output_shape)
    )

  def forward(self,x):
    x = self.conv_block_1(x)
    #print(x.shape)
    x = self.conv_block_2(x)
    #print(x.shape)
    x = self.classifier(x)
    return x
    #return self.classifier(self.conv_block_2(self.conv_block_1(x))) # benefits from operator fusion - speeds up computation on GPUs.


In [ ]:
torch.manual_seed(42)
model_0 = TinyVGG(input_shape=3, # no of color channels in the image data
                  hidden_units=10,
                  output_shape=len(class_names)).to(device)
model_0

In [ ]:
model_0.state_dict()

### 7.3 Trying forward pass on a single image to test the model

In [ ]:
# Create a image batch and a label batch
image_batch, label_batch = next(iter(train_dataloader_simple))

In [ ]:
# do the forward pass
model_0(image_batch.to(device))

### 7.4 Print model summary using `torchinfo`

In [ ]:
!pip install torchinfo

In [ ]:
from torchinfo import summary
model_summary = summary(model_0, input_size=[1,3,64,64])
model_summary

### 7.5 Creating training and testing loop functions

In [ ]:
def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device=device):
  # Put the model in train mode
  model.train()
  train_loss, train_acc = 0,0

  # loop through dataloader
  for batch, (X, y) in enumerate(dataloader):
    # send data to the target device
    X, y = X.to(device), y.to(device)

    y_pred = model(X) # output raw logits
    loss = loss_fn(y_pred, y)
    train_loss += loss.item()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Calculate the accuracy metric
    y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
    train_acc += ((y_pred_class == y).sum().item()/len(y_pred))

  # Return the adjusted loss and accuracy metrics
  train_loss = train_loss / len(dataloader)
  train_acc = train_acc / len(dataloader)
  return train_loss, train_acc

In [ ]:
def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device=device):
  # put model in eval mode
  model.eval()
  test_loss, test_acc = 0, 0

  with torch.inference_mode():
    for batch, (X,y) in enumerate(dataloader):
      X, y = X.to(device), y.to(device)
      test_pred = model(X)
      loss = loss_fn(test_pred, y)
      test_loss += loss.item()

      # Calculate the accuarcy metric
      test_pred_label = test_pred.argmax(dim=1)
      test_acc += ((test_pred_label == y).sum().item()/len(test_pred))

  # Return the adjusted loss and accuracy metrics
  test_loss = test_loss / len(dataloader)
  test_acc = test_acc / len(dataloader)
  return test_loss, test_acc

### 7.6 Write a `train` function to combine `train_step` and `test_step`

In [ ]:
from tqdm.auto import tqdm

# train function
def train(model: torch.nn.Module,
          train_loader: torch.utils.data.DataLoader,
          test_loader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          epochs: int=5,
          loss_fn: torch.nn.Module = nn.CrossEntropyLoss(),
          device=device):

  # Create an empty results dictionary
  results = {"train_loss": [],
             "train_accuracy": [],
             "test_loss": [],
             "test_accuracy": []}

  # Loop through the epochs
  for epoch in tqdm(range(epochs)):
    train_loss, train_acc = train_step(model,
                            train_loader,
                            loss_fn,
                            optimizer,
                            device)
    test_loss, test_acc = test_step(model,
                          test_loader,
                          loss_fn,
                          device)

    # Print out what's happening
    print(f"Epoch: {epoch} | Train_loss: {train_loss:.4f} | Train_acc: {train_acc:.4f} | Test_loss: {test_loss:.4f} | Test_acc: {test_acc:.4f}")

    # Update the results dictionary
    results["train_loss"].append(train_loss)
    results["train_accuracy"].append(train_acc)
    results["test_loss"].append(test_loss)
    results["test_accuracy"].append(test_acc)

  return results


In [ ]:
# Setup loss and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model_0.parameters(),
                             lr=0.001)

### 7.7 Training `model_0`

In [ ]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)

EPOCHS = 10 # hyperparam

# start the timer
from timeit import default_timer as Timer
start_time = Timer()

model_0 = TinyVGG(input_shape=3, hidden_units=10,
                  output_shape=len(class_names)).to(device)

model_0_results = train(model=model_0,
                        train_loader=train_dataloader_simple,
                        test_loader=test_dataloader_simple,
                        optimizer=optimizer,
                        epochs=EPOCHS,
                        loss_fn=loss_fn,
                        device=device)

end_time = Timer()
total_training_time = end_time - start_time
print(f"Total training time: {total_training_time:.3f} seconds")

In [ ]:
model_0_results

### 7.8 Plot the loss curves of model_0

**Loss curve** tracks a model's progress over time

In [ ]:
model_0_results.keys()

In [ ]:
def plot_loss_curve(results: Dict[str, List[float]]):
  """Plots training curves for a model results dictionary."""
  loss = results['train_loss']
  test_loss = results['test_loss']
  acc = results['train_accuracy']
  test_acc = results['test_accuracy']

  epochs = range(len(results['train_loss']))

  plt.figure(figsize=(14,6))
  plt.subplot(1,2,1)
  plt.plot(epochs, loss, label='train_loss')
  plt.plot(epochs, test_loss, label='test_loss')
  plt.ylabel("Loss")
  plt.xlabel("Epochs")
  plt.legend()

  plt.subplot(1,2,2)
  plt.plot(epochs, acc, label='train_accuracy')
  plt.plot(epochs, test_acc, label='test_accuracy')
  plt.ylabel("Accuracy")
  plt.xlabel("Epochs")
  plt.legend();

In [ ]:
plot_loss_curve(model_0_results)

## 8. Model 1: TinyVGG with Data Augmentation

### 8.1 Create transform with data augmentation

In [ ]:
train_transforms_trivial = transforms.Compose([
    transforms.Resize(size=(64,64)),
    transforms.TrivialAugmentWide(num_magnitude_bins=31),
    transforms.ToTensor()
])

test_transforms_simple = transforms.Compose([
    transforms.Resize(size=(64,64)),
    transforms.ToTensor()
])

### 8.2 Create train and test `datasets` and `dataloaders` with data augmentation

In [ ]:
# Create augmented datasets
train_data_augmented = datasets.ImageFolder(root=train_dir,
                                            transform=train_transforms_trivial,
                                            target_transform=None)

test_data_simple = datasets.ImageFolder(root=test_dir,
                                            transform=test_transforms_simple,
                                            target_transform=None)

len(train_data_augmented), len(test_data_simple)

In [ ]:
# Turn datasets into dataloaders
import os
BATCH_SIZE = 32
NUM_WORKERS = os.cpu_count()

torch.manual_seed(42) # we will shuffle the training data here
train_dataloader_augmented = DataLoader(dataset=train_data_augmented,
                                        batch_size=BATCH_SIZE,
                                        num_workers=NUM_WORKERS,
                                        shuffle=True)

test_dataloader_new = DataLoader(dataset=test_data_simple,
                                        batch_size=BATCH_SIZE,
                                        num_workers=NUM_WORKERS,
                                        shuffle=False)

### 8.3 Constructing and training Model 1


In [ ]:
# Create model 1
torch.manual_seed(42)
model_1 = TinyVGG(input_shape=3,
                  hidden_units=13,
                  output_shape=len(class_names)).to(device)
model_1

In [ ]:
# Train model 1 on augmented data
torch.manual_seed(42)
torch.cuda.manual_seed(42)

NUM_EPOCHS = 10 # hyperparam

# Setup loss function
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model_1.parameters(),
                             lr=0.001)

# Start the timer
from timeit import default_timer as Timer
start_time = Timer()

model_1_results = train(model=model_1,
                        train_loader=train_dataloader_augmented,
                        test_loader=test_dataloader_new,
                        optimizer=optimizer,
                        epochs=NUM_EPOCHS,
                        loss_fn=loss_fn,
                        device=device)

end_time = Timer()
total_training_time = end_time - start_time
print(f"Total training time: {total_training_time:.3f} seconds")

### 8.4 Plot the loss curves of `model_1`

In [ ]:
plot_loss_curve(model_1_results)

## 9. Compare model results

some ways to do this:
* hard coding
* MlFlow
* Weights and Biases

### 9.1 Using `Weights and Biases`

In [ ]:
import wandb
import torch
from torch import nn

# 1. Start a new run
run_model_0 = wandb.init(project="custom_datasets")
# 2. Save model inputs and hyperparameters
config_0 = run_model_0.config
config_0.dropout = 0.01
config_0.num_epochs = NUM_EPOCHS # Log the number of epochs
config_0.batch_size = BATCH_SIZE # Log the batch size
config_0.learning_rate = optimizer.param_groups[0]['lr'] # Log the learning rate

# 3. Log gradients and model parameters (optional, can be resource intensive)
# run.watch(model_1) # Keep commented out unless you specifically need to log gradients

# The loop below is for demonstrating batch processing, but logging metrics
# here without calculating them will cause an error.
# Since you are already calculating epoch-level metrics in your 'train' function,
# it is more efficient to log those results after training.
# Removing the erroneous batch logging loop:
# for batch_idx, (data, target) in enumerate(train_dataloader_augmented):
#   if batch_idx % log_interval == 0:
#    run.log({"loss": loss})
#    pass

# Log the epoch-level results after the training is complete
# Ensure model_1_results is available in this scope (it is from the previous cell)
if 'model_0_results' in locals():
    for epoch in range(len(model_0_results['train_loss'])):
        run_model_0.log({
            "epoch": epoch,
            "train_loss": model_0_results['train_loss'][epoch],
            "train_accuracy": model_0_results['train_accuracy'][epoch],
            "test_loss": model_0_results['test_loss'][epoch],
            "test_accuracy": model_0_results['test_accuracy'][epoch]
        })

print("Finished evaluating model 0")

# End the W&B run
run_model_0.finish()

run_model_1 = wandb.init(project="custom_datasets")
config_1 = run_model_1.config
config_1.dropout = 0.01
config_1.num_epochs = NUM_EPOCHS # Log the number of epochs
config_1.batch_size = BATCH_SIZE # Log the batch size
config_1.learning_rate = optimizer.param_groups[0]['lr']

if 'model_1_results' in locals():
    for epoch in range(len(model_1_results['train_loss'])):
        run_model_1.log({
            "epoch": epoch,
            "train_loss": model_1_results['train_loss'][epoch],
            "train_accuracy": model_1_results['train_accuracy'][epoch],
            "test_loss": model_1_results['test_loss'][epoch],
            "test_accuracy": model_1_results['test_accuracy'][epoch]
        })
print("Finished evaluating model 1")

run_model_1.finish()

### 9.2 Using hard coding

In [ ]:
import pandas as pd
model_0_df = pd.DataFrame(model_0_results)
model_1_df = pd.DataFrame(model_1_results)
model_0_df

In [ ]:
# setup a plot
plt.figure(figsize=(12,9))

# get no of epochs
epochs = range(len(model_0_df))

# Plot train loss
plt.subplot(2, 2, 1)
plt.plot(epochs, model_0_df['train_loss'], label='Model_0')
plt.plot(epochs, model_1_df['train_loss'], label='Model_1')
plt.title("Train Loss")
plt.xlabel("Epochs")
plt.legend();

# Plot test loss
plt.subplot(2, 2, 2)
plt.plot(epochs, model_0_df['test_loss'], label='Model_0')
plt.plot(epochs, model_1_df['test_loss'], label='Model_1')
plt.title("Test Loss")
plt.xlabel("Epochs")
plt.legend();

# Plot train acc
plt.subplot(2, 2, 3)
plt.plot(epochs, model_0_df['train_accuracy'], label='Model_0')
plt.plot(epochs, model_1_df['train_accuracy'], label='Model_1')
plt.title("Train Accuracy")
plt.xlabel("Epochs")
plt.legend();

# plot test acc
plt.subplot(2, 2, 4)
plt.plot(epochs, model_0_df['test_accuracy'], label='Model_0')
plt.plot(epochs, model_1_df['test_accuracy'], label='Model_1')
plt.title("Test Accuracy")
plt.xlabel("Epochs")
plt.legend();

## 10. Let's make predictions on a custom image now

In [ ]:
import requests

# Create path for custom img
custom_image_path = data_path / "Big_pizza.jpeg"

if not custom_image_path.is_file():
  with open(custom_image_path, "wb") as f:
    request = requests.get("https://raw.githubusercontent.com/Saloni0512/ImageData/main/Big_pizza.jpeg")
    f.write(request.content)
    print(f"Dowloading {custom_image_path}...")
else:
  print(f"{custom_image_path} already exist, skipping download")


### 10.1 Loading a custom image as tensor
* The image must be of size (64,64,3).
* The dtype of image tensor must be float32
* It must be on the same device

In [ ]:
custom_image_path

In [ ]:
import torchvision

# Read in image
custom_image_uint8 = torchvision.io.read_image(str(custom_image_path))
custom_image_uint8

In [ ]:
custom_image_uint8.shape, custom_image_uint8.dtype

In [ ]:
plt.imshow(custom_image_uint8.permute(1,2,0))

### 10.2 Making a prediction on the custom image with our trained model

In [ ]:
# load in the image and convert it to torch.float32
custom_image_tensor = torchvision.io.read_image(str(custom_image_path)).type(torch.float32) / 255
custom_image_tensor

In [ ]:
# Create a transform to reduce the shape of image
custom_transform = transforms.Compose([
    transforms.Resize(size=(64,64)),
])

custom_image_transformed = custom_transform(custom_image_tensor)
print(f"Transformed image shape: {custom_image_transformed.shape}")
plt.imshow(custom_image_transformed.permute(1,2,0))

In [ ]:
# We need to add a batch size here
model_1.eval()
with torch.inference_mode():
  custom_image_pred = model_1(custom_image_transformed.to(device))

In [ ]:
# This should run successfully
model_1.eval()
with torch.inference_mode():
  custom_image_pred = model_1(custom_image_transformed.unsqueeze(0).to(device))
custom_image_pred

> Note: Here, we get raw logits as predictions of the pizza, steak and sushi classes.

In [ ]:
# Convert logits into pred probs
custom_image_pred_probs = torch.softmax(custom_image_pred, dim=1)
custom_image_pred_probs

In [ ]:
# Convert pred probs into pred labels
custom_image_pred_label = torch.argmax(custom_image_pred_probs, dim=1)
custom_image_pred_label

In [ ]:
class_names[custom_image_pred_label]

### 10.3 Functionising the prediction of custom image